In [1]:
import pandas as pd
import numpy as np
import os
import glob
from fuzzywuzzy import process

C:\Users\s160555\AppData\Local\anaconda3\envs\geo_env\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#!pip install fuzzywuzzy

In [3]:
# Read the meta_data
#Z:\AT Data\Tableau Database\Archive\For Mintu
meta_data_path=pd.read_excel(r'Z:/AT Data/Tableau Database/Archive/For Mintu/D3/DATA LOCATION GUIDE.xlsx',sheet_name='D3')
meta_data_path.head()

,Intersection Description,ID,03 - Pedestrian Locations,03 - Bicycle Locations
0,HWY 49 & I80 EB off Ramp,3001,03P001 - 2023 - Hwy 49 & I80 EB Off Ramp,NaN
1,HWY 49 & Sawyer St,3002,03P002 - 2023 - Hwy 49 & Sawyer St,NaN
2,HWY 49 & Elm Ave/Fulweiler Ave,3003,03P003 - 2023 - Hwy 49 & Elm Ave_Fulwiler Ave,NaN
3,HWY 49 & Palm Ave,3004,03P004 - 2023 - Hwy 49 & Palm Rd,NaN
4,HWY 49 & Marguerite Mile Rd,3005,03P005 - 2023 - Hwy 49 & Marguerite Mile Rd,NaN


In [4]:
# now read all bike files within the folder
data_dir=r'Z:/AT Data/Tableau Database/Archive/For Mintu/D3/D3 Bicycle Sample'

# lets store the data
bike_data=glob.glob(os.path.join(data_dir,"*.csv"))
# store the data
agg=[]

# now iterate over different csv file
for file in bike_data:
    # extract file name
    filename=os.path.basename(file)
    # extract intersection name and direction
    try:
        name_part=filename.split("__")[1]
        name_parts = name_part.split('_')
        intersection_name=name_parts[1]
        direction = name_parts[2]
    except IndexError:
        print(f"Skipping invalid file name format: {filename}")
        continue     
        # Read CSV
    bdf = pd.read_csv(file)

    # Add extracted info to DataFrame
    bdf['Intersection'] = intersection_name
    bdf['Direction'] = direction

    # Append to list
    agg.append(bdf)

# Find common columns across all files
common_columns = set(agg[0].columns)
for df in agg[1:]:
    common_columns.intersection_update(df.columns)

# Combine all files using only common columns
com_df = pd.concat([df[list(common_columns)] for df in agg], ignore_index=True)
com_df=com_df.rename(columns={'Bike Lane':'Count'})
com_df=com_df.drop(['Unnamed: 2'],axis=1)

# Display the first few rows
com_df.head()

,Direction,Date,Count,Intersection
0,Eastbound,1/1/2023 0:00,0.0,Edgewood
1,Eastbound,1/1/2023 1:00,0.0,Edgewood
2,Eastbound,1/1/2023 2:00,0.0,Edgewood
3,Eastbound,1/1/2023 3:00,0.0,Edgewood
4,Eastbound,1/1/2023 4:00,0.0,Edgewood


In [5]:
# now join with intersection details 
if 'Intersection Description' in meta_data_path.columns:
    intersection_names=meta_data_path['Intersection Description'].tolist()

    def match_intersection(name):
        match, score = process.extractOne(name, intersection_names)  # Find best match
        return match if score > 80 else None 
            # Apply matching function to the final_df
    com_df['Matched_Intersection'] = com_df['Intersection'].apply(match_intersection)

    # Merge with intersection description data
    com_df = com_df.merge(meta_data_path, left_on='Matched_Intersection', right_on='Intersection Description', how='left')

else:
    print("Error: 'Intersection Description' column not found in the intersection description file.")
com_df.head()

,Direction,Date,Count,Intersection,Matched_Intersection,Intersection Description,ID,03 - Pedestrian Locations,03 - Bicycle Locations
0,Eastbound,1/1/2023 0:00,0.0,Edgewood,HWY 49 & Edgewood Rd,HWY 49 & Edgewood Rd,3006,03P006 - 2023 - Hwy 49 & Edgewood Rd,03B006 - 2023 - Hwy 49 & Edgewood Rd
1,Eastbound,1/1/2023 1:00,0.0,Edgewood,HWY 49 & Edgewood Rd,HWY 49 & Edgewood Rd,3006,03P006 - 2023 - Hwy 49 & Edgewood Rd,03B006 - 2023 - Hwy 49 & Edgewood Rd
2,Eastbound,1/1/2023 2:00,0.0,Edgewood,HWY 49 & Edgewood Rd,HWY 49 & Edgewood Rd,3006,03P006 - 2023 - Hwy 49 & Edgewood Rd,03B006 - 2023 - Hwy 49 & Edgewood Rd
3,Eastbound,1/1/2023 3:00,0.0,Edgewood,HWY 49 & Edgewood Rd,HWY 49 & Edgewood Rd,3006,03P006 - 2023 - Hwy 49 & Edgewood Rd,03B006 - 2023 - Hwy 49 & Edgewood Rd
4,Eastbound,1/1/2023 4:00,0.0,Edgewood,HWY 49 & Edgewood Rd,HWY 49 & Edgewood Rd,3006,03P006 - 2023 - Hwy 49 & Edgewood Rd,03B006 - 2023 - Hwy 49 & Edgewood Rd


In [6]:
# we will drop unnecessary columns
com_df1=com_df[['ID','Intersection Description','Direction','Date', 'Count']]
com_df2=com_df1.rename(columns={'ID':'Loc_id','Intersection Description':'Intersection'})
com_df2.head()

,Loc_id,Intersection,Direction,Date,Count
0,3006,HWY 49 & Edgewood Rd,Eastbound,1/1/2023 0:00,0.0
1,3006,HWY 49 & Edgewood Rd,Eastbound,1/1/2023 1:00,0.0
2,3006,HWY 49 & Edgewood Rd,Eastbound,1/1/2023 2:00,0.0
3,3006,HWY 49 & Edgewood Rd,Eastbound,1/1/2023 3:00,0.0
4,3006,HWY 49 & Edgewood Rd,Eastbound,1/1/2023 4:00,0.0


In [7]:
com_df2.Intersection.unique()

array(['HWY 49 & Edgewood Rd', 'HWY 50 & Bedford Ave',
       'HWY 50 & Canal St', 'HWY 50 & Spring St'], dtype=object)

In [8]:
# Now we will get the year, month, day, weekday and weekend status and also hour from data columns
# make sure the date is in date format to extract the temporal variable
com_df2['Date'] = pd.to_datetime(com_df2['Date'], format="%m/%d/%Y %H:%M")

# Extract year, month, day
com_df2['Year'] = com_df2['Date'].dt.year
com_df2['Month'] = com_df2['Date'].dt.strftime('%b') 
com_df2['Day'] = com_df2['Date'].dt.day_name()

# Extract weekday name and check if it's a weekend
com_df2['Weekday_Type'] = com_df2['Day'].apply(lambda x: "Weekend" if x in ['Saturday', 'Sunday'] else "Weekday")

# Extract hour and AM/PM
com_df2['Hour'] = com_df2['Date'].dt.strftime('%I %p').str.lstrip('0')
com_df2['District']=3
com_df2['Mode']="Bicycle"
com_df3=com_df2[['Loc_id','District','Intersection','Year', 'Month',
       'Day','Weekday_Type','Date', 'Hour','Mode','Direction','Count']]
com_df3.head()

,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Hour,Mode,Direction,Count
0,3006,3,HWY 49 & Edgewood Rd,2023,Jan,Sunday,Weekend,2023-01-01 00:00:00,12 AM,Bicycle,Eastbound,0.0
1,3006,3,HWY 49 & Edgewood Rd,2023,Jan,Sunday,Weekend,2023-01-01 01:00:00,1 AM,Bicycle,Eastbound,0.0
2,3006,3,HWY 49 & Edgewood Rd,2023,Jan,Sunday,Weekend,2023-01-01 02:00:00,2 AM,Bicycle,Eastbound,0.0
3,3006,3,HWY 49 & Edgewood Rd,2023,Jan,Sunday,Weekend,2023-01-01 03:00:00,3 AM,Bicycle,Eastbound,0.0
4,3006,3,HWY 49 & Edgewood Rd,2023,Jan,Sunday,Weekend,2023-01-01 04:00:00,4 AM,Bicycle,Eastbound,0.0


In [15]:
com_df2.shape

(72387, 12)

In [11]:
# Check the duplicate values and drop it
com_df4=com_df3.drop_duplicates(subset=['Loc_id','District','Intersection','Year', 'Month',
       'Day','Weekday_Type','Date', 'Hour','Mode','Direction','Count'])
com_df4.shape

(70899, 12)

In [12]:
# Save the file
com_df4.to_csv(("Z:/AT Data/Tableau Database/Archive/For Mintu/D3/output/combined_bike_data_with_desc.csv"), index=False)

# end of the script